In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/rsna-breast-cancer-detection/sample_submission.csv
/kaggle/input/rsna-breast-cancer-detection/train.csv
/kaggle/input/rsna-breast-cancer-detection/test.csv
/kaggle/input/rsna-breast-cancer-detection/train_images/10706/763186195.dcm
/kaggle/input/rsna-breast-cancer-detection/train_images/10706/937109986.dcm
/kaggle/input/rsna-breast-cancer-detection/train_images/10706/34700621.dcm
/kaggle/input/rsna-breast-cancer-detection/train_images/10706/1167990339.dcm
/kaggle/input/rsna-breast-cancer-detection/train_images/21867/1291014447.dcm
/kaggle/input/rsna-breast-cancer-detection/train_images/21867/831671840.dcm
/kaggle/input/rsna-breast-cancer-detection/train_images/21867/1481837831.dcm
/kaggle/input/rsna-breast-cancer-detection/train_images/21867/851000290.dcm
/kaggle/input/rsna-breast-cancer-detection/train_images/34867/1724313070.dcm
/kaggle/input/rsna-breast-cancer-detection/train_images/34867/1816992952.dcm
/kaggle/input/rsna-breast-cancer-detection/train_images/34867/1416

KeyboardInterrupt: 

In [9]:
import os
import numpy as np
import pandas as pd
import pickle
from pydicom import dcmread
import pylibjpeg
import cv2
from matplotlib import pyplot as plt 
from PIL import Image
import os.path

In [8]:
!pip install pylibjpeg pylibjpeg-libjpeg pydicom

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.3/4.3 MB 21.5 MB/s eta 0:00:0000:0100:01


In [2]:
path = "/kaggle/input/rsna-breast-cancer-detection/train_images"
dir_list = os.listdir(path)

In [3]:
df = pd.read_csv('/kaggle/input/rsna-breast-cancer-detection/train.csv')
df

,site_id,patient_id,image_id,laterality,view,age,cancer,biopsy,invasive,BIRADS,implant,density,machine_id,difficult_negative_case
0,2,10006,462822612,L,CC,61.0,0,0,0,NaN,0,NaN,29,False
1,2,10006,1459541791,L,MLO,61.0,0,0,0,NaN,0,NaN,29,False
2,2,10006,1864590858,R,MLO,61.0,0,0,0,NaN,0,NaN,29,False
3,2,10006,1874946579,R,CC,61.0,0,0,0,NaN,0,NaN,29,False
4,2,10011,220375232,L,CC,55.0,0,0,0,0.0,0,NaN,21,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
54701,1,9973,1729524723,R,MLO,43.0,0,0,0,1.0,0,C,49,False
54702,1,9989,63473691,L,MLO,60.0,0,0,0,NaN,0,C,216,False
54703,1,9989,1078943060,L,CC,60.0,0,0,0,NaN,0,C,216,False
54704,1,9989,398038886,R,MLO,60.0,0,0,0,0.0,0,C,216,True


In [10]:
((df['cancer']=='0').count())

54706

In [4]:
def dicom_to_png(ds,pos_neg):
    fn_name = ds.split('/')[-1][:-4]
    imagedata= pydicom.dcmread(ds)
    img = imagedata.pixel_array.astype(float)
    scaled_image = (np.maximum(img, 0) / img.max()) * 255.0
    scaled_image = np.uint8(scaled_image)
    final_image = Image.fromarray(scaled_image)
    parent_path='/kaggle/working/'
    sub_folder = pos_neg
    if not os.path.exists(parent_path):
        os.makedirs(parent_path)
    path = os.path.join(parent_path, sub_folder)

    if not os.path.exists(path):
        os.makedirs(path)
        final_image.save((path +'/{}.png').format(fn_name))
        #os.remove(ds)
        #cv2.imwrite((path +'/{}.png').format(fn_name),final_image)
    else:
        final_image.save((path +'/{}.png').format(fn_name))
        #os.remove(ds)
        #cv2.imwrite((path +'/{}.png').format(fn_name),final_image)
    #plt.imshow(img)
    #plt.show()

In [13]:
count_neg=0
count_pos=0

for i in dir_list:
    for j in os.listdir('/kaggle/input/rsna-breast-cancer-detection/train_images/'+i):
        if count_neg!=500:
            try:
                if ((df['image_id']==int(j[:-4]))& (df['cancer']==0)).any():
                    dicom_to_png('/kaggle/input/rsna-breast-cancer-detection/train_images/'+i+'/'+j,'negative')
                    count_neg=count_neg+1
            except:
                pass
        if count_pos!=500:
            try:
                if ((df['image_id']==int(j[:-4])) & (df['cancer']==1)).any():     
                    dicom_to_png('/kaggle/input/rsna-breast-cancer-detection/train_images/'+i+'/'+j,'positive')
                    count_pos=count_pos+1
            except:
                pass
                